# Football Betting using FiveThirtyEight's Soccer Power Index
## Goal
Backtest a strategy of using the FiveThirtyEight's Soccer Power Index for betting.

## Imports

In [1]:
import concurrent.futures
import io
import json
import os
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import requests

warnings.filterwarnings("ignore")

ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data")


## FiveThirtyEight
Download Soccer Power Index dataset.

In [2]:
SPI_SOCCER_URL = "https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv"

content = requests.get(SPI_SOCCER_URL, verify=False).content
df_spi = pd.read_csv(io.StringIO(content.decode()))
df_spi["date"] = pd.to_datetime(df_spi["date"], format="%Y-%m-%d")

assert df_spi["date"].isna().sum() == 0

df_spi.sample(5)


,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
2433,2016,2017-05-21,2411,Barclays Premier League,Arsenal,Everton,80.08,71.02,0.5950,0.1824,...,65.4,0.0,3.0,1.0,3.29,2.14,2.59,1.87,2.63,1.05
3366,2017,2017-08-19,1854,Italy Serie A,Verona,Napoli,46.39,81.53,0.1314,0.6605,...,42.7,56.5,1.0,3.0,1.53,2.56,0.77,2.67,0.84,3.15
2005,2016,2017-04-23,1854,Italy Serie A,Udinese,Cagliari,59.79,53.90,0.5123,0.2134,...,0.0,0.0,2.0,1.0,2.48,0.72,1.65,1.02,2.05,1.05
4568,2017,2017-09-29,1844,French Ligue 2,Paris FC,Nimes,30.49,39.69,0.3259,0.3685,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
13,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05


## Football-Data.co.uk
[football-data.co.uk](https://www.football-data.co.uk) is a website that provides historical betting odds for many soccer leagues.

In [3]:
FOOTBALL_DATA_MAIN_URL = "https://www.football-data.co.uk/mmz4281/{season}/{league}.csv"
LEAGUES = {
    "E0": ["Barclays Premier League"],
    "E1": ["English League Championship"],
    "E2": ["English League One"],
    "E3": ["English League Two"],
    "SC0": ["Scottish Premiership"],
    "D1": ["German Bundesliga"],
    "D2": ["German 2. Bundesliga"],
    "I1": ["Italy Serie A"],
    "I2": ["Italy Serie B"],
    "SP1": ["Spanish Primera Division"],
    "SP2": ["Spanish Segunda Division"],
    "F1": ["French Ligue 1"],
    "F2": ["French Ligue 2"],
    "N1": ["Dutch Eredivisie"],
    "B1": ["Belgian Jupiler League"],
    "P1": ["Portuguese Liga"],
    "T1": ["Turkish Turkcell Super Lig"],
    "G1": ["Greek Super League"],
}

FOOTBALL_DATA_OTHER_URL = "https://www.football-data.co.uk/new/{league}.csv"
OTHER_LEAGUES = {
    "ARG": ["Argentina Primera Division"],
    "AUT": ["Austrian T-Mobile Bundesliga"],
    "BRA": ["Brasileiro Série A"],
    "CHN": ["Chinese Super League"],
    "DNK": ["Danish SAS-Ligaen"],
    "JPN": ["Japanese J League"],
    "MEX": [
        "Mexican Primera Division Torneo Apertura",
        "Mexican Primera Division Torneo Clausura",
    ],
    "NOR": ["Norwegian Tippeligaen"],
    "RUS": ["Russian Premier Liga"],
    "SWE": ["Swedish Allsvenskan"],
    "SWZ": ["Swiss Raiffeisen Super League"],
}


def url_to_pandas(url):
    """Download URL content to a pandas dataframe."""
    content = requests.get(url, verify=False).content
    data = pd.read_csv(io.StringIO(content.decode(encoding="latin1")))
    data = data.dropna(how="all", axis=0)
    data = data.dropna(how="all", axis=1)
    data["URL"] = url
    return data


def get_football_data_main(year, league):
    """Get football data."""
    season = str(year - 1)[-2:] + str(year)[-2:]
    url = FOOTBALL_DATA_MAIN_URL.format(season=season, league=league)
    data = url_to_pandas(url)
    data["Season"] = season
    return data


def get_football_data_other(league):
    """Get football data."""
    url = FOOTBALL_DATA_OTHER_URL.format(league=league)
    data = url_to_pandas(url)
    data["Div"] = league
    return data


with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_football_data_main, year=year, league=league)
        for league in LEAGUES.keys()
        for year in range(2023, 2015, -1)
    ]
    results_main = [
        future.result() for future in concurrent.futures.as_completed(futures)
    ]

    futures = [
        executor.submit(get_football_data_other, league=league)
        for league in OTHER_LEAGUES.keys()
    ]
    results_other = [
        future.result() for future in concurrent.futures.as_completed(futures)
    ]

df_bet_main = pd.concat(results_main)
df_bet_other = pd.concat(results_other)


In [4]:
df_bet_main["MaxH"] = df_bet_main["MaxH"].fillna(df_bet_main["BbMxH"])
df_bet_main["MaxD"] = df_bet_main["MaxD"].fillna(df_bet_main["BbMxD"])
df_bet_main["MaxA"] = df_bet_main["MaxA"].fillna(df_bet_main["BbMxA"])
df_bet_main["AvgH"] = df_bet_main["AvgH"].fillna(df_bet_main["BbAvH"])
df_bet_main["AvgD"] = df_bet_main["AvgD"].fillna(df_bet_main["BbAvD"])
df_bet_main["AvgA"] = df_bet_main["AvgA"].fillna(df_bet_main["BbAvA"])

df_bet_other = df_bet_other.rename(
    columns={
        "Home": "HomeTeam",
        "Away": "AwayTeam",
        "HG": "FTHG",
        "AG": "FTAG",
        "Res": "FTR",
        "PH": "PSH",
        "PD": "PSD",
        "PA": "PSA",
    }
)


In [5]:
df_bet = pd.concat([df_bet_main, df_bet_other])


Dates comes in differents formats depending on the year.

In [6]:
date1 = pd.to_datetime(df_bet["Date"], format="%d/%m/%Y", errors="coerce")
date2 = pd.to_datetime(df_bet["Date"], format="%d/%m/%y", errors="coerce")
df_bet["Date"] = date1.fillna(date2)

assert df_bet["Date"].isna().sum() == 0

df_bet.sample(5)


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,HFKC,AFKC,Country,League
240,SP1,2017-02-28,NaN,Valencia,Leganes,1.0,0.0,H,1.0,0.0,...,23,-1.00,2.60,2.48,1.61,1.56,NaN,NaN,NaN,NaN
159,SP2,2016-11-20,NaN,Cadiz,Alcorcon,4.0,1.0,H,0.0,1.0,...,27,-0.25,2.05,1.97,1.90,1.83,NaN,NaN,NaN,NaN
104,SC0,2017-12-13,NaN,Partick,Motherwell,3.0,2.0,H,3.0,0.0,...,20,0.25,1.82,1.76,2.14,2.08,NaN,NaN,NaN,NaN
236,E3,2018-11-27,NaN,Yeovil,Port Vale,0.0,3.0,A,0.0,2.0,...,21,-0.25,2.15,2.03,1.85,1.78,NaN,NaN,NaN,NaN
106,E0,2017-11-05,NaN,Chelsea,Man United,1.0,0.0,H,0.0,0.0,...,20,-0.25,2.17,2.11,1.83,1.79,NaN,NaN,NaN,NaN


Each data source uses different ways of writing the same clubs names. I will use a dict to make names the same.

In [7]:
# from thefuzz import fuzz
# import networkx as nx

# # Generate empty names dict

# frames = [df_spi["team1"], df_spi["team2"], df_bet["HomeTeam"], df_bet["AwayTeam"]]
# names = pd.concat(frames).drop_duplicates()

# pairs = [
#     (name, other_name)
#     for name in names
#     for other_name in names
#     if fuzz.partial_ratio(name, other_name) > 90
# ]

# graph = nx.Graph()
# graph.add_edges_from(pairs)

# clusters = [list(cluster) for cluster in nx.connected_components(graph)]
# clusters = {cluster[0]: cluster for cluster in clusters}
# (
#     pd.Series(clusters)
#     .sort_index()
#     .to_json(os.path.join(DATA_DIR, "names.json"), force_ascii=False)
# )


In [8]:
with open(os.path.join(DATA_DIR, "names.json"), encoding="utf-8") as file:
    names_dict = json.load(file)


names_dict = {
    name.strip(): i.strip() for i, name_list in names_dict.items() for name in name_list
}


def clean(series, translate_dict):
    """Clean text in pandas series."""
    return series.str.strip().apply(
        lambda x: translate_dict[x] if x in translate_dict else x
    )


df_spi["home"] = clean(df_spi["team1"], names_dict)
df_spi["away"] = clean(df_spi["team2"], names_dict)
df_bet["home"] = clean(df_bet["HomeTeam"], names_dict)
df_bet["away"] = clean(df_bet["AwayTeam"], names_dict)


In [9]:
df_bet_before = df_bet.copy()
df_bet_after = df_bet.copy()

df_bet_before["Date"] = df_bet_before["Date"] - pd.Timedelta(days=1)
df_bet_after["Date"] = df_bet_after["Date"] + pd.Timedelta(days=1)

df_bet_expanded = pd.concat((df_bet, df_bet_before, df_bet_after))


In [10]:
leagues = [league for leagues in LEAGUES.values() for league in leagues]
other_leagues = [league for leagues in OTHER_LEAGUES.values() for league in leagues]

df_spi = df_spi.query(f"league in {leagues + other_leagues}").query(
    f"date < '{datetime.today().date()}'"
)


In [11]:
df_bet_expanded["dt"] = df_bet_expanded["Date"].dt.date
df_spi["dt"] = df_spi["date"].dt.date
df = df_spi.merge(
    df_bet_expanded,
    how="inner",
    on=["dt", "home", "away"],
    validate="1:1",
)
df = df.drop(columns=["HomeTeam", "AwayTeam", "team1", "team2"])


There are only a few clubs left that are unmatch. Maybe one dataset has more games than the other.

Now that names are fixed, it is able to be merged.

Add each results point of view to the dataset.

In [12]:
df_home = df.copy()
df_home["date"] = df["date"]
df_home["club"] = df["home"]
df_home["opponent"] = df["away"]
df_home["pov"] = "home"
df_home["outcome"] = df["FTR"] == "H"
df_home[">2.5"] = (df["FTHG"] + df["FTAG"]) > 2.5
df_home["<2.5"] = (df["FTHG"] + df["FTAG"]) < 2.5
df_home["prob"] = df["prob1"]
df_home["importance"] = df["importance1"]
df_home["xg"] = df["xg1"]
df_home["Avg"] = df["AvgH"]
df_home["Max"] = df["MaxH"]
df_home["B365"] = df["B365H"]
df_home["PS"] = df["PSH"]


In [13]:
df_away = df.copy()
df_away["date"] = df["date"]
df_away["club"] = df["away"]
df_away["opponent"] = df["home"]
df_away["pov"] = "away"
df_away["outcome"] = df["FTR"] == "A"
df_away[">2.5"] = (df["FTHG"] + df["FTAG"]) > 2.5
df_away["<2.5"] = (df["FTHG"] + df["FTAG"]) < 2.5
df_away["prob"] = df["prob2"]
df_away["importance"] = df["importance2"]
df_away["xg"] = df["xg2"]
df_away["Avg"] = df["AvgA"]
df_away["Max"] = df["MaxA"]
df_away["B365"] = df["B365A"]
df_away["PS"] = df["PSA"]


In [14]:
df_draw = df.copy()
df_draw["date"] = df["date"]
df_draw["club"] = df["home"]
df_draw["opponent"] = df["away"]
df_draw["pov"] = "draw"
df_draw["outcome"] = df["FTR"] == "D"
df_draw[">2.5"] = (df["FTHG"] + df["FTAG"]) > 2.5
df_draw["<2.5"] = (df["FTHG"] + df["FTAG"]) < 2.5
df_draw["prob"] = df["probtie"]
df_draw["importance"] = df[["importance1", "importance2"]].mean(axis=1)
df_draw["xg"] = df["xg1"] + df["xg2"]  # total
df_draw["Avg"] = df["AvgD"]
df_draw["Max"] = df["MaxD"]
df_draw["B365"] = df["B365D"]
df_draw["PS"] = df["PSD"]

In [15]:
df = pd.concat((df_home, df_away, df_draw))[
    [
        "date",
        "Div",
        "club",
        "opponent",
        "pov",
        "outcome",
        ">2.5",
        "<2.5",
        "prob",
        "importance",
        "xg",
        "Avg",
        "Max",
        "B365",
        "PS",
        "Avg>2.5",
        "Avg<2.5",
        "Max>2.5",
        "Max<2.5",
        "B365>2.5",
        "B365<2.5",
        "P>2.5",
        "P<2.5",
    ]
]

df.sample(5)

,date,Div,club,opponent,pov,outcome,>2.5,<2.5,prob,importance,...,B365,PS,Avg>2.5,Avg<2.5,Max>2.5,Max<2.5,B365>2.5,B365<2.5,P>2.5,P<2.5
2672,2017-08-12,E1,Brentford,Nottingham Forest,home,False,True,False,0.5008,NaN,...,1.85,1.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4208,2017-10-14,SP1,Real Sociedad,Alavés,away,True,False,True,0.3468,30.5,...,2.15,2.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,2016-12-03,SP1,Real Madrid,Barcelona,away,False,False,True,0.1803,100.0,...,4.20,4.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,2016-12-03,D1,Werder Bremen,FC Ingolstadt 04,home,True,True,False,0.4515,53.3,...,2.50,2.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2240,2017-06-11,BRA,Vitória,Atletico Mineiro,home,True,False,True,0.3855,NaN,...,NaN,2.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Results
### ROI Lines

In [31]:
THRESHOLD = 0.0
DIV = "SP1"


def expected_value(data, odds_col):
    """Calcula expected value."""
    return (data["prob"] * (data[odds_col] - 1)) - (1 - data["prob"])


def roi_lines(data, odds_col, threshold=0.0):
    """Calculate ROI."""
    ev = expected_value(data, odds_col)
    bet = ev > threshold
    balance = bet * (data["outcome"] * data[odds_col] - 1)
    return balance.sum() / bet.sum()


df_div = df.query(f"Div == '{DIV}'")


for bookmaker in ["Avg", "B365", "PS", "Max"]:
    roi = roi_lines(df_div, bookmaker, THRESHOLD)
    matches = expected_value(df_div, bookmaker)
    n_matches = matches.shape[0]
    n_bets = matches.loc[lambda x: x > THRESHOLD].shape[0]
    print(f"{bookmaker} ROI = {roi * 100:.2f}% (Bets = {100 * n_bets / n_matches:.2f}%)")


Avg ROI = -21.70% (Bets = 33.33%)
B365 ROI = -21.46% (Bets = 32.90%)
PS ROI = -11.85% (Bets = 42.15%)
Max ROI = -1.62% (Bets = 49.67%)


### ROI Over/Under

In [17]:
# def roi_over_under(data, odds_col):
#     """Calculate ROI."""
#     pd.options.mode.chained_assignment = None
#     data = data.copy()
#     over = (data["xg"] >= 2.5) * (data[">2.5"] * data[f"{odds_col}>2.5"] - 1)
#     under = (data["xg"] < 2.5) * (data["<2.5"] * data[f"{odds_col}<2.5"] - 1)
#     balance = over + under
#     return balance.sum()

# df_draw = df.query("pov == 'draw'")

# print(f"Avg ROI = {roi_over_under(df_draw, 'Avg') * 100:.2g}%")
# print(f"B365 ROI = {roi_over_under(df_draw, 'B365') * 100:.2g}%")
# print(f"Max ROI = {roi_over_under(df_draw, 'Max') * 100:.2g}%")


## Conclusion
### Lines
This strategy would lose money against the average betting site. However, it is able to have a small margin agains some specific websites that offers good odds.

### Over/Under
The over/under strategy is not profitable, even considering the best odds available.